In [91]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.utils import shuffle

In [92]:
df=pd.read_csv('/content/sample_data/letter-recognition.csv')
diag_map = {'A': 1.0, 'B': -1.0,'C': 1.0, 'D': -1.0,'E': 1.0, 'F': -1.0,'G': 1.0, 'H': -1.0,'I': 1.0, 'J': -1.0,'K': 1.0, 'L': -1.0,'M': 1.0, 'N': -1.0
            ,'O': 1.0, 'P': -1.0,'Q': 1.0, 'R': -1.0,'S': 1.0, 'T': -1.0,'U': 1.0, 'V': -1.0,'W': 1.0, 'X': -1.0,
            'Y': 1.0, 'Z': -1.0}
df['letter'] = df['letter'].map(diag_map)
df


,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,-1.0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,1.0,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,-1.0,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,-1.0,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,1.0,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,-1.0,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,1.0,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,-1.0,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,1.0,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [93]:
X=df.drop('letter',axis=1)
y=df['letter']
m=8000
X=X[:m]
y=y[:m]
print(X.shape)

(8000, 16)


In [94]:
df1=df
df1=df1.sample(frac=1)
X1=df1.drop('letter',axis=1)[:m]
y1=df1['letter'][:m]

In [95]:
tr_si=int(0.8*m)
X_train=X1[:tr_si]
y_train=y1[:tr_si]
X_test=X1[tr_si:]
y_test=y1[tr_si:]

In [96]:
print(X_train.shape,X_test.shape)

(6400, 16) (1600, 16)


In [97]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [98]:
y_pred=clf.predict(X_test)

**RESULTS SVM WITHOUT MARKOV SAMPLING**

In [99]:
print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_pred)))
print("recall on test dataset: {}".format(recall_score(y_test, y_pred)))
print("precision on test dataset: {}".format(recall_score(y_test, y_pred)))

accuracy on test dataset: 0.7275
recall on test dataset: 0.7836477987421384
precision on test dataset: 0.7836477987421384


In [100]:
N1=1000
X_val=X[:N1]
y_val=y[:N1]
print(y_val.shape)

(1000,)


In [101]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_val, y_val)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [102]:
y_pre=clf.predict(X_test)
print(y_pre)

[-1. -1. -1. ... -1. -1.  1.]


In [103]:
def Cost(Y_pred, Y):
    # calculate hinge loss
    if Y_pred*Y>1.0:
        return 0
    return 1.0-Y_pred*Y

In [104]:
X=X.to_numpy()
y=y.to_numpy()

In [105]:
Ni=0
j=0
k=2
q=1/1.2
print(q)
m=y.shape[0]
print(m)

0.8333333333333334
8000


In [106]:
import math
mp=0
mn=0
ct=0
X_tr=[]
y_tr=[]
X_te=[]
y_te=[]
X_t=X[0].reshape(1,-1)
print(X_t)
y_t=y[0].reshape(1,-1)[0]
X_tr.append(X_t[0])
y_tr.append(y_t[0])
if m%2==0 and y_t[0]==1.0:
    mp+=1
elif m%2==0 and y_t[0]==-1.0:
    mn+=1
pred_t=clf.predict(X_t)
print(pred_t)
p=1999999.0
for i in range(1,m):
    X_z=X[i].reshape(1,-1)
    y_z=y[i].reshape(1,-1)[0]
    pred_z=clf.predict(X_z)
    P=math.exp(-Cost(pred_z[0],y_z[0]))/math.exp(-Cost(pred_t[0],y_t[0]))
    if ct>=k:
        Pdd=q*P
        if Pdd>0.5:
            X_tr.append(X_z[0])
            y_tr.append(y_z[0])
            ct+=1 
            X_t=X_z
            y_t=y_z
            pred_t=pred_z
            Ni+=1
        else:
            X_te.append(X_z[0])
            y_te.append(y_z[0])
            ct=0
    elif P==1.0 and y_z==y_t:
        Pd=math.exp(-y_z[0]*pred_z[0])/math.exp(-y_t[0]*pred_t[0])
        if Pd>0.5:
            X_tr.append(X_z[0])
            y_tr.append(y_z[0])
            ct+=1
            X_t=X_z
            y_t=y_z
            pred_t=pred_z
            Ni+=1
        else:
            X_te.append(X_z[0])
            y_te.append(y_z[0])
            ct=0
    elif P==1.0 and y_z[0]*y_t[0]==-1.0 or P<1.0:
        Pd=math.exp(-y_z[0]*pred_z[0])/math.exp(-y_t[0]*pred_t[0])
        if Pd>0.5:
            X_tr.append(X_z[0])
            y_tr.append(y_z[0])
            ct+=1
            X_t=X_z
            y_t=y_z
            pred_t=pred_z
            Ni+=1
        else:
            X_te.append(X_z[0])
            y_te.append(y_z[0])
            ct=0
    elif P>1.0:
        X_tr.append(X_z[0])
        y_tr.append(y_z[0])
        ct+=1
        X_t=X_z
        y_t=y_z
        pred_t=pred_z
        Ni+=1
    if Ni>=N1:
        clf.fit(np.array(X_tr),np.array(y_tr))
        j+=1
    if j>=k:
        break

[[ 2  8  3  5  1  8 13  0  6  6 10  8  0  8  0  8]]
[-1.]


In [107]:
X_tr=np.array(X_tr)
y_tr=np.array(y_tr)
X_te=np.array(X_te)
y_te=np.array(y_te)

In [108]:
print(X_tr.shape[0],y_te.shape[0])

1002 207


In [109]:
clf = svm.SVC(kernel='linear')
clf.fit(X_tr, y_tr)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [110]:
y_pred=clf.predict(X_test)
# for i in range(X_te.shape[0]):
#     pre=clf.predict(X_te[i].reshape(1,-1))
#     y_pred.append(pre[0])
# y_pred=np.array(y_pred)

In [111]:
print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_pred)))
print("recall on test dataset: {}".format(recall_score(y_test, y_pred)))
print("precision on test dataset: {}".format(recall_score(y_test, y_pred)))

accuracy on test dataset: 0.7325
recall on test dataset: 0.7421383647798742
precision on test dataset: 0.7421383647798742
